
### Code for :
- #### Gradient descent
- #### Gradient descent implementation
- #### Multilayer Perceptron
- #### Backpropagation implementation


In [ ]:
# https://towardsdatascience.com/7-essential-tips-for-writing-with-jupyter-notebook-60972a1a8901#78aa
# https://github.com/dunovank/jupyter-themes#monospace-fonts-code-cells
# !pip install jupyterthemes
# !pip install --upgrade jupyterthemes

In [ ]:
# !jt -t oceans16
# !jt -t chesterish -tf ubuntu -tfs 13 -nf ptsans -nfs 14
# !jt -t chesterish -tf ubuntu -tfs 13
# !jt -t onedork -fs 95 -altp -tfs 18 -nfs 115 -cellw 88% -T
!jt -t onedork -tf ubuntu -tfs 15 -nf ptsans -nfs 18 -cellw 100%
# !jt -r

## Gradient Descent

As we discovered in few of our last sessions our first goal in creating neural network is to find the weights.
With these weights we want to make sure our model makes predictions as close as possible to real values.
But how would we will measure performance of model - here we will be using sum of squared errors (SSE)

Which can be represented with mathematical equation:
$E = \frac{1}{2}\sum_{\mu}\sum_{j}[{y_{j}}^{\mu}-{\hat{y}_{j}}^{\mu}]^{2}$

here $y$ is true value and $\hat{y}$ is predicted value

Now let's break and have a look at this equation

1. Inside sum is saying for each output unit, find the difference between the true value $y$ and the predicted value from the network $\hat{y}$ then square the difference, then sum up all those squares.
2. Outer sum is over $\mu$ is a sum over all the data points.

We are using SSE because The square ensures the error is always positive and larger errors are penalized more than smaller errors.

Remember that to minimize the squared Error we need to change weights of our model, thus we simply go back to __gradient descent__



We already know what Gradient Descent is - but here to oversimplify it I will say Gradient is another term for rate of change or slope.
To calculate rate of change we need to find derivatives of that point.

A derivative of a function gives us the slope
For example
$f(x) = x^2$
$f'(x) = 2x$
thus if $x =2$ the slope $f'(2) = 4$

One caveat of Gradient descent is that weights can be decreased to a point where error is low, but not the lowest, these spots are called local minima.
There are methods to mitigate this called momentum - you can read about it [here](https://distill.pub/2017/momentum/)

In [ ]:
!pip install -U pip
!pip install numpy

In [ ]:
import numpy as np


def sigmoid(x):
    """Calculate Sigmoid"""
    return 1/(1+np.exp(-x))

def sigmoid_prime(x):
    """Derivative of the sigmoid function"""
    return sigmoid(x) * (1 - sigmoid(x))

# We can create an array of input values
x = np.array([0.1, 0.3])

# Our expected output
y = 0.2

# Initial weights
w = np.array([-0.8, 0.5])

# The learning rate which should always be between 0-1
learnrate = 0.5

# the linear combination performed by the node (h in f(h) and f'(h))
# we take a dot product of our inputs and weights
h = np.dot(x, w)

# predicted output from our network (y-hat)
nn_output = sigmoid(h)

# output error (y - y-hat)
error = y - nn_output

# output gradient (f'(h))
output_grad = sigmoid_prime(h)

# error term (lowercase delta)
error_term = error * output_grad

# Gradient descent step
del_w = learnrate * error_term * x

print(f'Neural Network output: {nn_output} in percentage {nn_output:.1%}')
print(f'Amount of Error: {error} in percentage {error:.1%}')
print(f'Change in Weights: {del_w} ')

### Gradient descent implementation

Code above gives us an idea about how single weight update is implemented, but what about weight update on whole network.

Let us take another example which uses school admissions data from UCLA.
This dataset has three inputs
GRE score, GPA and rank of undergrad school (where rank 1 is for highest prestige and 4 is lowest)

Our goal is to predict old age question - if student should be admitted to graduate program or not.


#### Data cleanup

After checking the CSV file, we see that there are 4 columns "admit", "gre", "gpa", "rank"
Now the rank data is not in normalized for here - so let's break them down into dummy variables/columns with 0 and 1
Besides this gre and gpa needs also to be prepared - both need to have zero mean and SD of 1.
This step is important because of our sigmoid function which if you remember reduces very small and very large values near to zero, which in turn make our descent go to zero too.

#### Mean Square Error

To get our error calculated we will make a change from sum of squared errors to mean of squared errors, becuase we are using a large dataset and summing all of it will result in a
large change for gradient descent. We can do compensation by using a very small learning rate but that won't be effective alternative.
Therefore, we will be taking mean and keeping our learning rate in range of 0.01 to 0.001

Now mean square error can be equated as :

$E = \frac{1}{2m}\sum_{\mu}[{y^{\mu}-{\hat{y}^{\mu}]^{2}$

Generally these steps are followed for updating weights through gradient descent:

1. set the weight step to zero $\Delta w_i = 0$
2. loop through each record in training data:
        1. Move forward through network and calculate output $\hat{y} = f(\sum_iw_ix_i)$
        2. Calculate the error term for the output unit $\delta = (y - \hat{y}) * f'(\sum_iw_ix_i)$
        2. update the weight step $Δw_i = Δw_i + \delta x_i$
3. Update the weights $w_i = \frac{w_i + \eta \Delta w_i}{m}$ here $\eta$ is learning rate and $m$ is number of records.
4. Repeat for $e$ epochs




$E=\frac{1}{2m}\sum_{\mu}[{y^{\mu}-{\hat{y}^{\mu}]^{2}$

In [77]:
import numpy as np
import pandas as pd

## read csv into a var
admissions = pd.read_csv('binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
# remove the rank column
data = data.drop('rank', axis=1)

# Standardize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std

# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.iloc[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']


## Implementing the Gradient Descent here
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

# TODO: We haven't provided the sigmoid_prime function like we did in
#       the previous lesson to encourage you to come up with a more
#       efficient solution. If you need a hint, check out the comments
#       in solution.py from the previous lecture.

# Use to same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape
print(f'n_records: {n_records}')
print(f'n_features: {n_features}')
last_loss = None

# Initialize weights randomly so that they are diverged and asymmetrical
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Epochs or iterations and learnrate are known as hyperparameters of neural network
epochs = 1000
learnrate = 0.5

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    # We want to loop through all the data keeping x as input, y as target
    for x, y in zip(features.values, targets):

        # np.dot is used to get product of two arrays - which is our h activation function
        output = sigmoid(np.dot(x, weights))

        # The error, the target minus the network output
        error = y - output

        # instead of creating another function we calculate faster here and store in a reusable var
        error_term = error * output * (1 - output)


        # The gradient descent step, the error times the gradient times the inputs
        del_w += error_term * x

    # Update the weights here. The learning rate times the
    # change in weights, divided by the number of records to average
    weights += learnrate * del_w / n_records
    # print(f"Weights: {weights}")

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print(f"Prediction accuracy: {accuracy:.3f} in percentage {accuracy:.2%}")


n_records: 360
n_features: 6
Train loss:  0.2627609384996635
Train loss:  0.20928619409324875
Train loss:  0.20084292908073426
Train loss:  0.19862156475527873
Train loss:  0.1977985139668603
Train loss:  0.19742577912189868
Train loss:  0.1972350774624106
Train loss:  0.1971294562509248
Train loss:  0.19706766341315082
Train loss:  0.19703005801777368
Prediction accuracy: 0.725 in percentage 72.50%


### Multilayer Perceptron
#### Implementing the hidden layer

Now we need to deal with multiple input and multiple hidden units, thus the weights between them will require
two indices $w_ij$ where $i$ denotes input units and $j$ are hidden units.



In [78]:
import numpy as np

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1/(1+np.exp(-x))

# Network size
N_input = 4
N_hidden = 3
N_output = 2

np.random.seed(42)
# Make some fake data
X = np.random.randn(4)

weights_input_to_hidden = np.random.normal(0, scale=0.1, size=(N_input, N_hidden))
weights_hidden_to_output = np.random.normal(0, scale=0.1, size=(N_hidden, N_output))


# TODO: Make a forward pass through the network

hidden_layer_in = np.dot(X, weights_input_to_hidden)
hidden_layer_out = sigmoid(hidden_layer_in)

print(f'Hidden-layer Output: {hidden_layer_out}')

output_layer_in = np.dot(hidden_layer_out, weights_hidden_to_output)
output_layer_out = sigmoid(output_layer_in)

print(f'Output-layer Output: {output_layer_out}')


Hidden-layer Output: [0.41492192 0.42604313 0.5002434 ]
Output-layer Output: [0.49815196 0.48539772]


#### Implementing backpropagation

The backpropagation algorithm is just an extension of weight update using gradient descent, where we use chain rule to find the errors with respect to weights connecting hidden and input layers.

Let's implement the algorithm for graduate admission school data.
We want to
- Implement the forward pass.
- Implement the backpropagation algorithm.
- Update the weights.


In [89]:
#### ------ DATA PREP --------- ####
import numpy as np
import pandas as pd

## read csv into a var
admissions = pd.read_csv('binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
# remove the rank column
data = data.drop('rank', axis=1)

# Standardize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std

# Split off random 10% of the data for testing
np.random.seed(80)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.iloc[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

#### ------ IMPLEMENTATION --------- ####

# remember selecting seed also is important - if you change the seed you will difference in accuracy.
# np.random.seed(25)

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))


# Hyperparameters
n_hidden = 2  # number of hidden units
epochs = 900
learnrate = 0.005

n_records, n_features = features.shape
last_loss = None
# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5, size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5, size=n_hidden)

for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        ## Forward pass ##
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)

        output = sigmoid(np.dot(hidden_output, weights_hidden_output))

        ## Backward pass ##
        error = y - output
        output_error_term = error * output * (1 - output)

        ## propagate errors to hidden layer
        hidden_error = np.dot(output_error_term, weights_hidden_output)
        hidden_error_term = hidden_error * hidden_output * (1 - hidden_output)

        del_w_hidden_output += output_error_term * hidden_output
        del_w_input_hidden += hidden_error_term * x[:, None]


    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output, weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print(f"Prediction accuracy: {accuracy:.3f} in percentage: {accuracy:.2%}")


Train loss:  0.29135986165965394
Train loss:  0.2893218729151472
Train loss:  0.2873191667944074
Train loss:  0.28535246721671065
Train loss:  0.283422431956647
Train loss:  0.28152964950581233
Train loss:  0.2796746365249894
Train loss:  0.27785783590588
Train loss:  0.2760796154508361
Train loss:  0.2743402671686853
Prediction accuracy: 0.425 in percentage: 42.50%
